In [31]:
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from data.dataset import LaneDataset
from models.lanenet.lanenet import LaneNet

from common.utils import get_project_root
import os

In [32]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"training on {DEVICE}")

training on cpu


In [33]:
root = get_project_root()
train_path = os.path.join(root, "data", "train_set_example")

log_dir = os.path.join(root, "experiments/exp0")
writer = SummaryWriter(log_dir=log_dir)

training_data = LaneDataset(train_path)
train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True)

In [34]:
epochs = 1
model = LaneNet([3, 16, 64, 128, 64, 16], 4).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for e in range(epochs):
    print(f"epoch {e+1}/{epochs}")
    total_loss = 0.0
    total_seg_loss = 0.0
    total_inst_loss = 0.0
    
    for idx, batch in enumerate(train_dataloader):
        img, bin_label, inst_label = batch
        
        img = img.to(DEVICE)
        bin_label = bin_label.to(DEVICE)
        inst_label = inst_label.to(DEVICE)
        
        optimizer.zero_grad()
        bin_pred, inst_pred = model(img)
        
        loss, seg_loss, inst_loss = model.compute_loss((bin_pred, inst_pred), (bin_label, inst_label))
        loss.backward()
        optimizer.step()
        
        
        total_loss += loss.item()
        total_seg_loss += seg_loss.item()
        total_inst_loss += inst_loss.item()
        writer.add_scalar("loss", total_loss, e)
        writer.add_scalar("seg_loss", total_seg_loss, e)
        writer.add_scalar("inst_loss", total_inst_loss, e)

writer.flush()
writer.close()
    
    

epoch 1/1


ValueError: not enough values to unpack (expected 3, got 2)